# 전국 스타벅스 매장 정보 가져오기

# 라이브러리 로드

In [ ]:
# 라이브러리 로드
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup as bs
from tqdm import tqdm_notebook
from selenium import webdriver
# 웹 화면이 로딩될 때까지 기다려주는 모듈
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# HTML 파일 읽어오기
html 파일 읽어오기(requests라이브러리, BeautifulSoup라이브러리)

In [ ]:
# 웹드라이버 설정
chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument("headless")

driver = webdriver.Chrome("**********************************", options=chrome_options)
# 페이지 전체가 로딩되는 시간까지 10초
driver.implicitly_wait(10)

# 스타벅스 매장 지도 url
starbucks_map_url = "https://www.starbucks.co.kr/store/store_map.do?disp=locale"
driver.get(starbucks_map_url)

In [ ]:
# 스타벅스 서울
starbucks_seoul_css = "#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(1) > a"
WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, starbucks_seoul_css))).click()

## 스타벅스 서울 매장 위치 HTML 읽어오기

In [ ]:
# 스타벅스 서울 전체 클릭
starbucks_seoul_all_css = "#mCSB_2_container > ul > li:nth-child(1) > a"
WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, starbucks_seoul_all_css))).click()

In [ ]:
# 스타벅스 서울 전체 HTML 파싱
html = driver.page_source
soup = bs(html, "lxml")

In [ ]:
li_all = soup.select("#mCSB_3_container > ul > li")
li_all[0]

In [ ]:
starbucks_data = []

for li in li_all:
    name = li.find("strong").text.strip()
    address = li.find("p").text.strip().replace("1522-3232", "")
    city = address.split(" ")[0]
    gu = address.split(" ")[1]
    load = address.split(" ", 2)[2]
#     print(name, address, gu, sep=" | ")
    each = {
        "매장이름": name,
        "시": city,
        "구": gu,
        "도로명주소": load
    }
    starbucks_data.append(each)

In [ ]:
len(starbucks_data)

In [ ]:
df_starbucks = pd.DataFrame(starbucks_data)
df_starbucks.tail()

In [ ]:
df_starbucks.describe()

In [ ]:
df_starbucks.info()

# 파일로 저장하기

In [ ]:
# 저장할 파일명
file_name = "starbucks_seoul_local.csv"
# CSV 파일로 저장하기
df_starbucks.to_csv(file_name, index=False)
# 저장한 파일 확인
pd.read_csv(file_name)